In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import silhouette_score

# Function to extract features using a pre-trained CNN model
def extract_features(frame, model):
    # Preprocess the frame for the VGG16 model
    preprocessed_frame = preprocess_input(frame.astype(np.float32))
    # Expand dimensions to match VGG input shape (batch size of 1)
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
    # Extract features from the last convolutional layer (block5_conv3) of VGG16
    features = model.predict(preprocessed_frame)
    return features.flatten()

# Function to read video and extract features from each frame
def extract_frame_features(video_path, model):
    cap = cv2.VideoCapture(video_path)
    features = []
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        features.append(extract_features(frame, model))
    cap.release()
    return frames, np.array(features)

# Function to save segmented clips
def save_segmented_clips(video_path, frames, cluster_labels):
    for cluster_id in np.unique(cluster_labels):
        cluster_indices = np.where(cluster_labels == cluster_id)[0]
        start_frame = cluster_indices[0]
        end_frame = cluster_indices[-1]
        segmented_clip = frames[start_frame:end_frame + 1]
        out_path = f'/content/drive/MyDrive/experiment/S10_VGG_16/cluster_{cluster_id}_segment.mp4'
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frames[0].shape[1], frames[0].shape[0]))
        for frame in segmented_clip:
            out.write(frame)
        out.release()
        print(f'Saved segmented clip for cluster {cluster_id} to {out_path}')

# Apply K-means clustering
def segment_video(video_path, num_clusters, model):
    frames, frame_features = extract_frame_features(video_path, model)
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(frame_features)
    cluster_labels = kmeans.labels_

    # Post-processing to merge adjacent frames in the same cluster
    merged_labels = np.copy(cluster_labels)
    for i in range(1, len(merged_labels)):
        if merged_labels[i] == merged_labels[i - 1]:
            merged_labels[i] = -1

    save_segmented_clips(video_path, frames, merged_labels)
    return frame_features, cluster_labels

# Load pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

# Run segmentation
video_path = '/content/drive/MyDrive/experiment/s10.mp4'  # Update with your video path
num_clusters = 15  # Update with the desired number of clusters
frame_features, cluster_labels = segment_video(video_path, num_clusters, model)
silhouette_avg = silhouette_score(frame_features, cluster_labels)
print(f"Silhouette Score for KMeans clustering: {silhouette_avg}")


1/1 [==============================] - 0s 33ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Saved segmented clip for cluster -1 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_-1_segment.mp4
Saved segmented clip for cluster 0 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_0_segment.mp4
Saved segmented clip for cluster 1 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_1_segment.mp4
Saved segmented clip for cluster 2 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_2_segment.mp4
Saved segmented clip for cluster 3 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_3_segment.mp4
Saved segmented clip for cluster 4 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_4_segment.mp4
Saved segmented clip for cluster 5 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_5_segment.mp4
Saved segmented clip for cluster 6 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_6_segment.mp4
Saved segmented clip for cluster 7 to /content/drive/MyDrive/experiment/S10_VGG_16/cluster_7_segment.mp4
Saved segmented clip for cluster 8 to /content/drive/

In [ ]:
result_dict = {"s6": 0.3588101267814636, "s7": 0.41046321392059326, "s8" : 0.38652530312538147, "s9": 0.3401716947555542}